# Trabajo Final de la materia Minería de Datos I

## Olimpia Saucedo Estrada y Francisco Ramírez Castañeda

### Preprocesamiento de Datos
Debido a que el conjunto de muestras contiene un mayor número de muestras maliciosas es necesario realizar un balanceo de estas mismas, por lo que se obtuvieron muestras maliciosas aleatoriamente del dataset ajustando el numero de muestras a 400 para posteriormente concatenar las muestras benignas. Con esto se obtuvo un conjunto de muestras balanceado.

Con el conjunto de muestras balanceado se realizó el preprocesamiento de los datos mediante los algoritmos TF-IDF y Word2Vec, los cuales permiten la extración de características para poder realizar el entrenamiento de los modelos de clasificación.

In [29]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.pipeline import Pipeline
import features
from sklearn.linear_model import Perceptron




In [5]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /home/olibits/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
dataset = pd.read_csv('dataset.csv')
df_minoria = dataset[dataset.y == 1]
df_mayoria = dataset[dataset.y == 0]
df_minoria_resample = resample(df_minoria, replace=True, n_samples=400, random_state=123)
dataset_balanceado = pd.concat([df_mayoria, df_minoria_resample])


In [14]:
#2 Caracterizar las muestras utilizando TF-IDF 
X,Y = dataset_balanceado['X'],dataset_balanceado['y']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
vec = TfidfVectorizer()


In [15]:
#2 Caracterizar las muestras utilizando Word2Vec
model_gn = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)
w2v = dict(zip(model_gn.index2word, model_gn.vectors))
X_tokens = [word_tokenize(s) for s in X]
X_trainw2v, X_testw2v, Y_trainw2v, Y_testw2v = train_test_split(X_tokens, Y, test_size=0.2, random_state=0)

In [30]:
help(Perceptron)

Help on class Perceptron in module sklearn.linear_model.perceptron:

class Perceptron(sklearn.linear_model.stochastic_gradient.BaseSGDClassifier)
 |  Perceptron(penalty=None, alpha=0.0001, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, eta0=1.0, n_jobs=None, random_state=0, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False)
 |  
 |  Perceptron
 |  
 |  Read more in the :ref:`User Guide <perceptron>`.
 |  
 |  Parameters
 |  ----------
 |  
 |  penalty : None, 'l2' or 'l1' or 'elasticnet'
 |      The penalty (aka regularization term) to be used. Defaults to None.
 |  
 |  alpha : float
 |      Constant that multiplies the regularization term if regularization is
 |      used. Defaults to 0.0001
 |  
 |  fit_intercept : bool
 |      Whether the intercept should be estimated or not. If False, the
 |      data is assumed to be already centered. Defaults to True.
 |  
 |  max_iter : int, optional (default=1000)
 | 

In [32]:
perceptron = Perceptron()
perceptron_vec = Pipeline([('vectorizer', vec), ('pac', perceptron)])

perceptron_vec.fit(X_train, Y_train)
perceptron_score_vec = perceptron_vec.score(X_test, Y_test)
perceptron_score_vec

perceptron_w2v = Pipeline([
    ("word2vec vectorizer", features.MeanEmbeddingVectorizer(w2v)),
    ("svm", svc)])
perceptron_vec.fit(X_train, Y_train)
perceptron_score_vec = perceptron_vec.score(X_test, Y_test)
perceptron_score_vec

0.9361702127659575

In [27]:
#TODO 3 Entrenar los algoritmos (Hay que dividirnos esta parte)
#maquina de soporte vectorial tfidf
svc = svm.SVC(C=10)
svm_clf = Pipeline([('vectorizer', vec), ('pac', svc)])
svm_clf.fit(X_train, Y_train)
svc_score = svm_clf.score(X_test, Y_test)
print(svc_score)

#maquina de soporte vectorial w2v
svm_w2v = Pipeline([
    ("word2vec vectorizer", features.MeanEmbeddingVectorizer(w2v)),
    ("svm", svc)])

svm_w2v.fit(X_trainw2v, Y_trainw2v)
svc_w2v_score=svm_w2v.score(X_testw2v, Y_testw2v)
print(svc_w2v_score)

/home/olibits/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8617021276595744


/home/olibits/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9042553191489362


In [ ]:
#TODO 4 Comparar los resultados

In [ ]:
#TODO 5 Graficar la matriz de confusión y curva ROC del mejor algoritmo

### Conclusiones